# Skull AI

The objective here is to make AI/Bots that can play the deception board/tile game: Skull. See this link [LINK] to a video showing how to play and then the flow diagram in the gitrepo showing the decision flow for  a player.

ASSUMPTIONS:
 - Only 4 players in a game
 - for now, wont pay attention to previous rounds
 - simple terms (linear)
 - max bet to place is 12

In [1]:
import random
from numpy.random import rand, choice
import numpy as np
import pandas as pd
from copy import deepcopy
from math import log
from time import time

In [21]:
class skullPlayer:
    """
    The main class that handles all functions and actions that a player themselves would need to take. 
    """
    def __init__(self):
        """
        The initialiser of this class.
        """
        self.flowers = 3       #GR          how many flowers the player currently has
        self.skull = 1         #GR          how many skulls the player currently has
        self.totalTiles = 4    #GR          how many tiles the player has in total
        self.inPlay = True     #GR          is the player still in the game 
        self.hasPoint = False  #GR          does the player have a point already
        self.externalHasSkull = True  #GR*  do the other players know for certain that the player has a skull
        self.externalHasNoSkull = False #GR do the other player know the player does not have a skull
        self.placedTiles = []  #RR          what tiles the player has currently placed
        self.hasPassed = False #RR          has the the player passed in this round of betting
        self.currentBet = None #RR          the current bet the player has made
        self.verbose = False
        
        self.recPosition = None #GR
        self.gameState = None #GR
        self.betrec = None #RR*
        self.game = None #GR
        self.pickRec = None #RR*
        
        self.age=1
        self.winTwo=0
        self.winElim=0
        
        self.theta_tp=(rand(1,30)*2)-1 #AI
        self.theta_bp=(rand(1,31)*2)-1 #AI
        self.theta_w2b=(rand(12,95)*2-1) #AI
        self.theta_bpa=(rand(1,95)*2-1) #AI
        self.theta_w2p=(rand(3,93)*2-1) #AI
        
    def removeTile(self):
        """
        Remove a tile from the players available tile. If there is a choice it will do so randomly.
        """
        if self.inPlay:
            if self.skull==0:
                self.flowers+=-1
            else:
                if choice([True]+[False]*self.flowers):
                    self.skull+=-1
                else:
                    self.flowers+=-1

            self.totalTiles+=-1
            self.externalHasSkull = False

            if self.totalTiles==0:
                self.inPlay=False
                self.gameState.updateFirstPlayer((self.recPosition+1)%4)
                self.game.activePlayer = self.recPosition
            else:
                self.gameState.updateFirstPlayer(self.recPosition)
                self.game.activePlayer = (self.recPosition-1)%4
            self.game.resetRound()
        else:
            raise NameError("Player is not in play")
                
    def placeFlower(self):
        """
        Adds a flower, if available, to the players list of placed tiles
        """
        if self.inPlay & (self.flowers>0):
            if (len(self.placedTiles)-sum(self.placedTiles))<self.flowers:
                self.placedTiles.append(False)
                self.gameState.gatherInfo()
            else:
                raise NameError('All flowers have been placed')
        else:
            if not(self.inPlay):
                raise NameError("Player is not in play")
            else:
                raise NameError('No flowers to place')
    
    def placeSkull(self):
        """
        Adds a skull, if available, to the players list of placed tiles
        """
        if self.inPlay & (self.skull>0):
            if any(self.placedTiles):
                raise NameError('Skull has been placed')
            else:
                self.placedTiles.append(True)
                self.gameState.gatherInfo()
        else:
            if not(self.inPlay):
                raise NameError("Player is not in play")
            else:
                raise NameError('No Skull to place')
                
    def resetRound(self):
        """
        resets the variables within the player that needs to be reset for a new round
        """
        self.placedTiles=[]
        self.hasPassed = False
        self.currentBet = None
        
    def resetGame(self):
        """
        resets the variables within the player that needs to be reset for a new game
        """
        self.flowers=3
        self.skull=1
        self.totalTiles=4
        self.inPlay=True
        self.hasPoint = False
        self.externalHasSkull = True
        self.externalHasNoSkull = False
        
        self.recPosition = None
        self.gameState = None
        self.game = None
        self.betrec = None
        self.pickRec = None        
        self.verbose = False
        
        self.resetRound()
        
    def playerSummary(self):
        """
        prints basic information about the player
        """
        print("Flowers: {}\nSkull: {}\nPlaced tiles: {}".format(self.flowers,self.skull, self.placedTiles))
        
    def placeTile(self):
        """
        Given the player has decided to place a tile go ahead and place a tile. If the choice is available it will make the choice, if not it will place the option that is available to it.
        """
        if self.inPlay & (len(self.placedTiles)<self.totalTiles):
            if any(self.placedTiles)|(self.skull==0):
                self.placeFlower()
            elif ((len(self.placedTiles)-sum(self.placedTiles))==self.flowers):
                self.placeSkull()
            else:
                self.chooseTileToPlace()
        else:
            if not(self.inPlay):
                pass
            else:
                raise NameError('Max number of tiles placed')
            
    def chooseTileToPlace(self):
        """
        Based on the information available, given the player has decided to place a tile and has the choice available, decide what tile to place.
        """
        if self.inPlay:
            inVec=np.reshape(self.gameState.state,(-1,1))
            inVec=np.concatenate((inVec,np.array([[rand()],[1]])))

            skull = ((1/(1+np.exp(-np.dot(self.theta_tp,inVec))))>=0.5)[0][0]

            if skull:
                self.placeSkull()
            else:
                self.placeFlower()
        else:
            raise NameError("Player is not in play")
            
    def betOrPlace(self): 
        """
        Based on the information available, decide wether to make a bet or place a tile.
        """
        if self.inPlay:
            if len(self.placedTiles)<self.totalTiles:
                inVec=np.reshape(self.gameState.state,(-1,1))
                inVec=np.concatenate((inVec,np.array([[int(any(self.placedTiles))],[rand()],[1]])))

                betOrPlace = ((1/(1+np.exp(-np.dot(self.theta_bp,inVec))))>=0.5)[0][0]

                if betOrPlace:
                    bet = self.whatToBet()
                    if self.verbose:
                        print("Player bets: {}".format(bet))
                    self.currentBet=bet
                    self.betrec.playerbet(self.recPosition,bet)
                    self.game.stage+=1
                else:
                    self.placeTile()
                    if self.verbose:
                        print("Tile placed")
            else:
                bet = self.whatToBet()
                if self.verbose:
                    print("Player bets: {}".format(bet))
                self.currentBet=bet
                self.betrec.playerbet(self.recPosition,bet)
                self.game.stage+=1
        else:
            pass
            
            
    def whatToBet(self):
        """
        Based on the information available, and given that the player has decided to bet, decide what value to bet.
        """
        if self.inPlay:
            inVec=np.concatenate((np.reshape(self.gameState.state,(-1,1)),
                                  np.reshape(self.betrec.betRecord,(-1,1)),np.array([[int(any(self.placedTiles))],[rand()],[1]])))
            availableBets=np.zeros((1,12))
            availableBets[:,0:sum(np.array(self.gameState.state)[:,2])]=1
            currentBet=(np.where(self.betrec.betRecord[:,:-1].any(axis=0))[0])
            if currentBet.shape[0]==0:
                currentBet=0
            else:
                currentBet=max(currentBet)+1
            availableBets[:,0:currentBet]=0
            inVec=np.concatenate((inVec,availableBets.T))
            bet=np.argmax((1/(1+np.exp(-np.dot(self.theta_w2b,inVec))))*availableBets.T)+1
            self.bet=bet
            return bet
        else:
            raise NameError("Player is not in play")
    
    def betOrPass(self):
        """
        Based on the information available, decide to bet or pass.
        """
        if self.inPlay & (self.game.stage==0) & (self.hasPassed==False):
            inVec=np.concatenate((np.reshape(self.gameState.state,(-1,1)),np.reshape(self.betrec.betRecord,(-1,1)),
                                  np.array([[int(any(self.placedTiles))],[rand()],[1]])))
            availableBets=np.zeros((1,12))
            availableBets[:,0:sum(np.array(self.gameState.state)[:,2])]=1
            currentBet=(np.where(self.betrec.betRecord.any(axis=0))[0])
            if currentBet.shape[0]==0:
                currentBet=0
            else:
                currentBet=max(currentBet)+1
            availableBets[:,0:currentBet]=0
            if availableBets.any():
                inVec=np.concatenate((inVec,availableBets.T))
                betOrPass=(1/(1+np.exp(-np.dot(self.theta_bpa,inVec))))>=0.5

                if betOrPass:
                    bet = self.whatToBet()
                    if self.verbose:
                        print("Player bets: {}".format(bet))
                    self.currentBet=bet
                    self.betrec.playerbet(self.recPosition,bet)

                else:
                    self.hasPassed=True
                    if self.verbose:
                        print("Player passes")
                    self.betrec.playerPass(self.recPosition)
                    if self.betrec.switch:
                        self.game.stage+=1
            else:
                self.hasPassed=True
                if self.verbose:
                    print("Player passes")
                self.betrec.playerPass(self.recPosition)
                if self.betrec.switch:
                        self.game.stage+=1
        else:
            if not(self.inPlay):
                pass
            elif not(self.game.stage==0):
                raise NameError('Not in the betting stage')
            
    def takeTurn(self):
        """
        Given the current stage of the game, have the player do their turn
        """
        if self.inPlay & (self.hasPassed==False):
            if self.game.stage==-1:
                self.betOrPlace()
            elif self.game.stage==0:
                self.betOrPass()
            elif self.game.stage==1:
                self.pickRec.setPickRec(self.recPosition)
                self.seek()
        elif self.verbose:
            if self.hasPassed:
                print("Player {} has passed this round".format(self.recPosition))
            else:
                print("Player {} is out of the game".format(self.recPosition))
            
    def seek(self):
        """
        Engages the seek proceedure if the player has the highest bet of the round
        """
        if self.inPlay & (self.game.stage==1) & (self.hasPassed==False):
            if any(self.placedTiles):
                if self.verbose:
                    print("Player {} skulls themself".format(self.recPosition))
                self.removeTile()
            else:
                toFind = self.currentBet - len(self.placedTiles)
                idList = list(range(0,self.recPosition))+list(range(self.recPosition+1,4))
                while toFind>0:
                    rowId = self.whoToPick()
                    pickedPlayer = idList[rowId]
                    if self.verbose:
                        print("picked player {}".format(pickedPlayer))
                    tile = self.pickRec.pickPlayer(pickedPlayer,rowId)
                    if tile:
                        break
                    else:
                        toFind+=-1
                        if self.verbose:
                            print("{} flowers left to find".format(toFind))
                if toFind>0:
                    if self.verbose:
                        print("player has been skulled")
                    self.removeTile()
                else:
                    if self.verbose:
                        print("player {} has won a point".format(self.recPosition))
                    if self.hasPoint:
                        if self.verbose:
                            print("player {} wins the game with 2 points".format(self.recPosition))
                        self.winTwo+=1
                        self.game.winnerFound=True
                        self.game.winner=self
                    else:
                        self.hasPoint=True
                        self.game.activePlayer = (self.recPosition-1)%4
                        self.gameState.updateFirstPlayer(self.recPosition)
                        self.game.resetRound()
                        
                

    def whoToPick(self):
        """
        Based on the information available and the if the player is seeking, decides which player tile to turn over. 
        """
        inVec=np.concatenate((np.reshape(self.gameState.state,(-1,1)),np.reshape(self.betrec.betRecord,(-1,1)),
                              np.reshape(self.pickRec.pickRecord,(-1,1)),np.array([[1]])))
        whoToPick = np.argmax((1/(1+np.exp(-np.dot(self.theta_w2p,inVec))))*np.array(self.pickRec.pickRecord)[:,-1].reshape(3,1))
        return whoToPick
    
    def mutate(self,learningRate=0.0001):
        """
        Perturb the coefficents of the variables used to make the decisions.
        :param learningRate: float, the learning rate to control the pertubation.
        """
        self.age+=1
        self.theta_tp+=(rand(1,30)*2-1)*learningRate #AI
        self.theta_bp+=(rand(1,31)*2-1)*learningRate #AI
        self.theta_w2b+=(rand(12,95)*2-1)*learningRate #AI
        self.theta_bpa+=(rand(1,95)*2-1)*learningRate #AI
        self.theta_w2p+=(rand(3,93)*2-1)*learningRate #AI
        
    def genSummary(self):
        """
        Print a quick summary of the player with respect to its generation information.
        """
        print("Player age: {}\nPlayer 2 point wins: {}\nPlayer Elimination wins: {}".format(self.age,
        self.winTwo,
        self.winElim))

In [15]:
class skullGame:
    """
    The main class for the game itself
    """
    
    def __init__(self,player1,player2,player3,player4,verbose=False):
        """
        The initialiser of this class.
        :param player1,player2,player3,player4: skullPlayer, the players participating in this game. 
        """
        self.playerArray = [player1,player2,player3,player4]
        self.firstPlayerOfRound = random.randint(0,len(self.playerArray)-1)
        self.activePlayer=self.firstPlayerOfRound
        self.gameState = skullGameState(self.firstPlayerOfRound,*self.playerArray)
        self.betRec = skullBetRec(*self.playerArray)                               #RR
        self.stage = -1                                                            #RR
        self.pickRec = skullPickRec(*self.playerArray)                             #RR
        self.verbose = verbose
        self.winnerFound=False
        self.winner=None
        for i in range(0,4):
            player = self.playerArray[i]
            player.recPosition=i
            player.gameState = self.gameState
            player.betrec = self.betRec
            player.game = self
            player.placeTile()
            player.pickRec = self.pickRec
            player.verbose = self.verbose
    
    def resetRound(self):
        """
        Will reset the players and the internal information that needs to be reset for a new round
        """
        self.betRec = skullBetRec(*self.playerArray)
        self.stage = -1
        self.gameState.gatherInfo()
        self.pickRec = skullPickRec(*self.playerArray)
        for i in range(0,4):
            player = self.playerArray[i]
            player.betrec = self.betRec #potentially redundant
            player.pickRec = self.pickRec
            player.resetRound()
            
        for i in range(0,4):
            player = self.playerArray[i]
            player.placeTile()
        if self.verbose:
            display(self.gameState.state)
            print("\n")
            
    def playGame(self):
        """
        Begins the game and plays it out until a winner is found
        """
        self.activePlayer = self.firstPlayerOfRound
        while not(self.winnerFound):
            if sum(np.array(self.gameState.state)[:,-1])==3:
                winId = np.argmin(np.array(self.gameState.state)[:,-1])
                self.winnerFound=True
                self.winner=self.playerArray[winId]
                self.playerArray[winId].winElim+=1
                if self.verbose:
                    print("player {} wins the game by player elimination".format(winId))
                break
            if self.verbose:
                print("player {}'s turn".format(self.activePlayer))
            self.playerArray[self.activePlayer].takeTurn()
            self.activePlayer=(self.activePlayer+1)%4

In [16]:
class skullGameState:
    """
    A class to handle and manage the game state information through the games and the rounds as they are played out. For each player it shall have:
        - If they have a point (binary)
        - How many tiles they have (numerical)
        - How many tiles they have placed (numerical)
        - If they have their skull (dual-binary) [1,0] skull, [0,0] unknown, [0,1] no skull 
        - their place in the turn order (numerical)
        - if they're out of the game (binary) [1] out, [0] in
    """
    
    def __init__(self,firstPlayer,player1,player2,player3,player4):
        """
        The initialiser of this class.
        :param firstPlayer: int, the record id of the player who is first in this current round
        :param player1,player2,player3,player4: skullPlayers, that are in the game
        """
        self.playerArray = [player1,player2,player3,player4]
        self.firstPlayer = firstPlayer
        self.state=[]
        self.gatherInfo()
        
        
    def gatherInfo(self):
        """
        Gather all the gatestate information at the point this is activated
        """
        self.state=[]
        for i in range(0,4):
            player = self.playerArray[i]
            self.state.append([int(player.hasPoint),player.totalTiles,len(player.placedTiles),
                               int(player.externalHasSkull),int(player.externalHasNoSkull),
                               ((i-self.firstPlayer)%4),1-int(player.inPlay)])
    
    def updateFirstPlayer(self,player):
        """
        Update the internal refference who is the first player
        :param player: int, the record id of who is the first player
        """
        self.firstPlayer = player

In [17]:
class skullBetRec:
    """
    A class to store and handle the betting record for a round of the game. For each player it has an indicatior vector for each bet they have made and an additional entry for if the player has passed
    """
    def __init__(self,player1,player2,player3,player4):
        """
        The initialiser of this class.
        :param player1,player2,player3,player4: skullPlayer, the players that are in the given game
        """
        self.playerArray = [player1,player2,player3,player4]
        self.betRecord=np.zeros((len(self.playerArray),(len(self.playerArray)*3)+1))
        self.switch = False
        for i in range(0,4):
            player = self.playerArray[i]
            self.betRecord[i,-1] = (1-int(player.inPlay))
            
    def playerbet(self,playerId,bet):
        """
        Update the bet record with the bet the given player has just made
        :param playerId: int, the id of the player who is making the bet
        :param bet: int, the bet the player has actually made
        """
        self.betRecord[playerId,bet-1] = 1
    
    def playerPass(self,playerId):
        """
        Update the bet record with the given player having just passed
        :param playerId: int, the id of the player who is passing
        """
        self.betRecord[playerId,-1] = 1
        if sum(self.betRecord[:,-1])==3:
            self.switch=True

In [18]:
class skullPickRec:
    """
    A class to store and handle the record of the picking information when a player is seeking their flowers. for it other player it'll have: 
        - if they have been previously picked
        - how many tiles they have remaining
        - how many tiles have been selected already from them
        - if the player is actually eligible to pick from
    """
    def __init__(self,player1,player2,player3,player4):
        """
        The initialiser of this class.
        :param player1,player2,player3,player4: skullPlayer,the players that are in the given game
        """
        self.pickerId=None
        self.playerArray = [player1,player2,player3,player4]
        self.pickRecord=[]
        
    def setPickRec(self,pickId):
        """
        Sets the id of the player that is picking and then collects the initial information that is available to the seeker.
        :param pickId: int, the id of the player that will be seeking.
        """
        self.pickerId=pickId
        for i in range(0,4):
            if i==self.pickerId:
                continue
            else:
                player = self.playerArray[i]
                self.pickRecord.append([0,len(player.placedTiles),0,int(player.inPlay)])
    
    def pickPlayer(self,playerId,rowId):
        """
        Handles the picking of another player when seeking.
        :param playerId: int, the id of the picked player in the player array 
        :param rowId: int the row id of the picked player in picking record
        """
        pickedPlayer = self.playerArray[playerId]
        tile = pickedPlayer.placedTiles.pop
        self.adjustInfo(rowId)
        
        if tile:
            pickedPlayer.externalHasSkull=True
        elif self.pickRecord[rowId,2] ==pickedPlayer.totalTiles:
            pickedPlayer.externalHasNoSkull=True
        return tile
    
    def adjustInfo(self,rowId):
        """
        Update the pick record when a player has been picked.
        :param rowId: int, the row id in the pick record to update.
        """
        self.pickRecord[rowId][0] = 1
        self.pickRecord[rowId][1] += -1
        self.pickRecord[rowId][2] += 1
        if self.pickRecord[rowId][1]==0:
            self.pickRecord[rowId][3]=0

In [28]:
class skullTrainer:
    """
    A class to do and handle the training of the players which are auto-generate. 
    """
    
    def __init__(self,passThrough=4,noOfChild=205,newBorns=204,generations=100):
        """
        The initialiser of this class.
        :param passThrough: int, how many players should survive to be parents of the next generation 
        :param noOfChild: int, how many childeren each parent should make. Including themselves
        :param newBorns: int, how many new children from no parents should be added
        :param generations: int, how many generations the training should go through.
        """
        self.passThrough = passThrough
        self.noOfChild = noOfChild
        self.newBorns = newBorns
        self.generations = generations
        self.totalPop = (passThrough*noOfChild)+newBorns
        self.noOfGames = int(log(self.totalPop,self.passThrough))-1
        self.populationList=[skullPlayer() for x in range(0,self.totalPop)]
        self.gamesList=[]
        
    
    def cleanUp(self):
        """
        Will run through the current population list and perform a game reset on them so they are ready for the next game. The population list is shuffled afterwards to that the players play other from different parents/origins
        """
        for player in self.populationList:
            player.resetGame()
        random.shuffle(self.populationList)
    
    def nextGen(self):
        """
        Creates the next generation from the current population list. A copy of the parent is included as well fresh "newborns". At the end the new population list is shuffled to ensure players play other players.
        """
        nextGenList=[]
        for player in self.populationList:
            nextGenList.append(player)
            for i in range(0,self.noOfChild-1):
                playerCopy=deepcopy(player)
                playerCopy.mutate()
                nextGenList.append(playerCopy)
        nextGenList+=[skullPlayer() for x in range(0,self.newBorns)]
        self.populationList=nextGenList
        random.shuffle(self.populationList)
    
    def createGames(self):
        """
        Will take the current population of the generation and will create games for each group of 4 and add them to the list of current games.
        """
        for i in range(0,len(self.populationList)//4):
            self.gamesList.append(skullGame(*self.populationList[i*4:i*4+4]))
    
    def playGames(self):
        """
        Will go through the games and play them. Once complete it will add the winners to a list. Upon all games being completed the games list will be wipped and the winnersList will replace the current population list.
        """
        winnerList=[]
        for game in self.gamesList:
            game.playGame()
            winnerList.append(game.winner)
        self.gamesList=[]
        self.populationList=winnerList
    
    def train(self,verbose=True):
        """
        Begins the training proceedure for training the AI via genetic evolution.
        :param verbose: boolean, to control wether to print updates as the training proceeds 
        """
        currentGen=1
        while currentGen<=self.generations:
            if verbose:
                print("Current Generation: {}".format(currentGen))
            t2=time()
            for gameNo in range(0,self.noOfGames):
                if verbose:
                    print("Current round: {}".format(gameNo+1),end="\r")
                self.createGames()
                self.playGames()
                self.cleanUp()
            if (currentGen!=self.generations):
                if verbose:
                    print("Creating next generation", end="\r")
                self.nextGen()
            if verbose:
                print("Generation completed after: {} S".format(time()-t2))
            currentGen+=1
        print("Training Complete")
            
    def survivorsSummary(self):
        """
        Will print the gerneration summary of each player remaining in the population list.
        """
        for player in self.populationList:
            player.genSummary()

In [65]:
# demoPlayers=[skullPlayer() for x in range(0,4)]
# demoGame=skullGame(*demoPlayers,True)
# demoGame.playGame()
# display(demoGame.gameState.state)

In [66]:
demoTrain=skullTrainer(generations=500)
demoTrain.train(verbose=False)

Training Complete


In [ ]:
demoTrain.survivorsSummary()

In [ ]:
demoTrain.populationList

In [69]:
# import pickle
# file_to_store = open("winningPlayers1.pickle", "wb")
# pickle.dump(demoTrain.populationList, file_to_store)
# file_to_store.close()

In [70]:
# file_to_read = open("winningPlayers1.pickle", "rb")
# loaded_object = pickle.load(file_to_read)
# file_to_read.close()

In [ ]:
trainedGame=skullGame(*demoTrain.populationList,True)
trainedGame.playGame()